In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
import a_mps, b_model, c_tebd, tfi_exact

In [ ]:
L = 14
J = 1.0
g = 1.5
up = a_mps.init_spinup_MPS(L)
m = b_model.TFIModel(L=L, J=J , g=g)
print(f"sigma z: {up.site_expectation_value(m.sigmaz)}")
print(f"sigma x: {up.site_expectation_value(m.sigmax)}")

In [ ]:
right = a_mps.init_spinright_MPS(L)
print(f"sigma z: {right.site_expectation_value(m.sigmaz)}")
print(f"sigma x: {right.site_expectation_value(m.sigmax)}")

In [ ]:
L = 14
J = 1
gs = [0.5, 1.0, 1.5]
up = a_mps.init_spinup_MPS(L)
right = a_mps.init_spinright_MPS(L)

for g in gs:
    m = b_model.TFIModel(L=L, J=J, g=g)
    print(f"{g = :.1f}: {m.energy(up) = :.2f}")
    print(f"{g = :.1f}: {m.energy(right) = :.2f}")

In [ ]:
%run c_tebd

In [ ]:
def global_quench(L: int, dt: float, chi_max: int, eps: float) -> tuple[list[float], list[float], list[float]]:
    """Real time evoulution. returns (time, magnetization, entropy)"""
    J = 1.0
    g = 1.5
    N_steps = round(10*J / dt)
    m = b_model.TFIModel(L=L, J=J, g=g)
    up = a_mps.init_spinup_MPS(L)
    bonds = c_tebd.calc_U_bonds(m, 1j * dt)
    def calc_M(mps):
        return sum(mps.site_expectation_value(m.sigmaz))
    def calc_S(mps):
        return mps.entanglement_entropy()[L//2]
    ts = [0]
    Ms = [calc_M(up)]
    Ss = [calc_S(up)]
    for state in c_tebd.iterate_TEBD(up, bonds, N_steps, chi_max, eps):
        ts.append(ts[-1] + dt)
        Ms.append(sum(state.site_expectation_value(m.sigmaz)))
        Ss.append(state.entanglement_entropy()[L//2])
    return ts, Ms, Ss


In [ ]:
dts = np.logspace(-1, -4, 4)
default_dt = 1e-1
chis = [10, 15, 20, 30, 50]
default_chi = 30
Ls = [14, 50]
results: dict[tuple[int, float, float], tuple[list[float], list[float], list[float]]] = {}
for L in Ls:
    for chi in chis:
        results[L, chi, default_dt] = global_quench(L=L, chi_max=chi, dt=default_dt, eps=1e-10)
    for dt in dts:
        results[L, default_chi, dt] = global_quench(L=L, chi_max=default_chi, dt=dt, eps=1e-10)

In [ ]:
plt.figure()

plt.subplot(2, 2, 1)
L = Ls[0]
dt = default_dt
for chi in chis:
    t, M, S = results[(L, chi, dt)]
    plt.plot(t, M, label=f"{chi = }")
plt.title(f"{L = }")

plt.subplot(2, 2, 2)
L = Ls[1]
dt = default_dt
for chi in chis:
    t, M, S = results[(L, chi, dt)]
    plt.plot(t, M, label=f"{chi = }")
plt.title(f"{L = }")

plt.subplot(2, 2, 3)
L = Ls[0]
chi = default_chi
for dt in dts:
    t, M, S = results[(L, chi, dt)]
    plt.plot(t, M, label=f"{dt = }")
plt.title(f"{L = }")

plt.subplot(2, 2, 4)
L = Ls[1]
chi = default_chi
for dt in dts:
    t, M, S = results[(L, chi, dt)]
    plt.plot(t, M, label=f"{dt = }")
plt.title(f"{L = }")

plt.show()